<a href="https://colab.research.google.com/github/thad75/OptionAI/blob/main/TDm/SR/RecSysTD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-20m.zip
!unzip "/content/ml-20m.zip"
!pip install gdown
!gdown --id 1b9XN-AD2ph6Akdy4gpWdx8rIlK6HZ-u9
!pip install pytorch-lightning
import os
import random
import shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pycocotools
import pytorch_lightning as pl
import scipy.sparse as sparse
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from IPython.display import clear_output, display
from pytorch_lightning import loggers as pl_loggers
from scipy.sparse import coo_matrix
from scipy.sparse.csgraph import laplacian
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch import nn
from torch.nn import Module
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torchvision.datasets import MNIST
import torchmetrics
import matplotlib.pyplot as plt

pd.set_option("expand_frame_repr", False)


# Disclaimer 

These are interviews questions from companies such as PMU, Amazon, Netflix. Try not to use ChatGPT as much as possible as ChatGPT is not, yet, allowed in interviews.
Don't hesitate to ask questions ! We are paid for that. A live correction will be given.

# Goal of this lab


*   Apply Recommendation Systems to Industrial Datasets
*   Get to know Pytorch-Lightning
*  **Be lost**
* Discover new frameworks and read documentation
* Explore Datas and think



# Hands-On I : Recommending an artist

Given the following data, we want to recommend a list of artists from our existing database to the listener. 

*  Using similarity computation, recommend in the decreasing order the artists 
that suits listerners' taste

In [16]:
data = {
    'Artist': ['Celine Dion', 'Fred Again', 'Elton John', 'Black Eyed Peas', 'BTS', 'Anirudh', 'A.R Rahman', 'User'],
    'International': [1, 1, 1, 1, 1, 1, 1, 1],
    'Sings in French': [1, 0, 0, 0, 0, 0, 0, 0],
    'Versatility': [0, 0, 0, 0, 0, 1, 1, 1],
    'Danceability': [0, 1, 0, 1, 1, 1, 1, 0],
    'Makes Film Music (BGM)': [0, 0, 1, 0, 0, 1, 1, 1],
    'Won Awards': [1, 1, 1, 1, 1, 1, 1, 1],
    'Recent Hit': [0, 1, 0, 0, 1, 1, 1, 0]
}

df = pd.DataFrame(data)
df = df.set_index('Artist')

# TODO : Print the dataframe


In [ ]:
from sklearn.metrics.pairwise import manhattan_distances

# TODO : Extract users' likings (check loc method from df)
user= ...
other_line = ...

# TODO : Extract all keys except the 'User' keys
keys = [... for key in df.index ...]

# TODO : Define a Similarity Function (Cosine or Distance will be enough)
# Hint : Look at what we loaded for ya *wink wink*
def similarity_function(l1, l2):
  return ...

# TODO : Compare each artists to the user's taste and print the list of recommendable artist.
values = []
for key in keys: 
  value = ...
  values.append((key, value))
  
distance_df = pd.DataFrame(values, columns=["key", "similarity"])
distance_df.set_index('key')

print(distance_df)

# TODO : Conclude


# Hands-On II : Matrix Factorization


## I - Load, Analyze, Transform Amazon Luxury

As future Data Engineer, Data Analyst or Data Scientist, you will stumble upon the usual ETLs pipeline, that most companies lack in. As you might knowk, whether its for analysis or training, if garbage comes in, garbage goes out. So always, take time to correctly analyze your data.

<img src="https://swordstem.com/wp-content/uploads/2019/04/372.png">

Useful doc: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html


We will be using an Industrial Dataset: Amazon Luxury. 

### a - Load

Obviously, in this case the load step is not complex. But in your career, you will surely stumble on SQL, GraphQL, Neo4j, ElasticSearch.. databases, that are in an absolute mess.

In [ ]:
# Loading /content/ratings_Beauty.csv into a dataframe

df = pd.read_csv("/content/ratings_Beauty.csv")

### b - Analyze


The data is loaded into a Pandas Dataframe. Let's analyze and understand the dataset.


* What are the Dataframes keys ?
* How many items are there ? How many users are there ?
* How many unique items are there ? How many unique users are there ?
* What's the span of the ratings ?
* Plot the number of Ratings that are present. Is it a Gaussian Distribution ? A Skewed Gaussian Distribution ? 
* What could it mean on the quality of the products ? 

In [ ]:
# TODO : Perform the asked Data Exploration using pandas

num_items = ...
num_users = ...
rating_span = ...

# TODO : Plot the Distribution using plt


### c - Transform

* Using pivot methods, transform your dataframe to a Rating Matrix. If your session crashes, pass this question. You'll create other Rating Matrix later.
* Process the Nan Values of your Rating Matrix

In [ ]:
# We are limiting the dataset to the first 50 lines due to the limitation in Google Colab
df_mf = df.head(50)

# TODO : Create your rating matrix
rating_matrix = ...

# Print your rating matrix, what do you observe ?
rating_matrix

In [ ]:
# TODO : Replace all the Nan values with 0. Google what you need, the needed function is a method on Numpy
def correct_outliers(R):
    return ...

rating_matrix_processed = correct_outliers(rating_matrix)
# TODO : Plot the processed rating matrix


## II - Matrix Factorization


### a - Quick Theory Recap

The good thing about Matrix Factorization is its usability for either content based recommendation or collaborative filtering based recommendation. This is one of the earliest model used by Netflix. 

Matrix Factorization finds latent structure in data such as it decomposes a Feedback Matrix $A\in \mathbb{R}^{M*N}$ with M the number of users and n the number of items, into :     
* $P$ :  a user embedding matrix $\in \mathbb{R}^{M*K}$
* $Q$ : an item embedding matrix $\in \mathbb{R}^{N*K}$

The embeddings are learned such that 
$ \hat{A} = P*Q^{T} \approx A$

We define the Objective Function as:
<div align=center >
$\underset{P \in \mathbb{R}^{M \times K}, Q \in \mathbb{R}^{N \times K}}{\text{minimize}}\ \underset{(i,j)\in\text{obs}}{\sum} (A_{ij} - \hat{A}_{ij})^2$
over $P$ and  $Q$</div>

To update the values of the matrixes, we use the typical Gradient Descent. In fact, each element of P and Q are updated using the following equations :

$p_{ik} = p_{ik} + 2\alpha*error* q_{kj}$

$q_{kj} = q_{kj} + 2\alpha*error* p_{ik}$




* Where do the $p_{ik}$ and $q_{kj}$ update formula come from ?

### b - The Code

Now, we are going to code the Matrix Factorization Class that will perform matrix factorization for us.

The class has 4 methods :
* the init : for attribute initialization
* the error : to compute the error
* the update : to update the elements of P and Q 
* the run : to perform the calcultation for a num_step of times

In [ ]:
class Matrix_Factorization():
  """
  This class performs Matrix Factorization for a given FeedBack Matrix
  """

  def __init__(self,A, latent_dim, steps= 5000, learning_rate = 0.0002):

      self.A = A
      self.latent_dim = latent_dim
      self.steps = steps
      self.learning_rate = learning_rate

      # TODO : Initialize the correct attributes
      # M,N are the shape of the FeedBack Matrix
      self.M, self.N  = ...

      # TODO : Initialize two random matrix P and Q. Don't forget to Transpose Q
      self.P = ...
      self.Q = ...

  def error(self, a, b):
      # TODO : Return the error : the difference between a and b
      return ...

  def update(self):
    # TODO : Perform element update using gradient descent
    # TODO : Iterate among the user
    for user in range(...):
        # TODO : Iterate among the items
        for item in range(...):
          if self.A[user][item]> 0:
            # TODO : Compute the error between the GT Rating and the Predicted Rating of user on item
            error = ...
            # TODO : Using the gradient descent formula update the values of the elements of P and Q
            for latent in range(self.latent_dim):
                self.P[user][latent] = ...
                self.Q[latent][item] = ...

  
  def run(self):   
    # TODO : Call the update method 'steps' times   
    for step in range(self.steps) : 
        ...
    return np.dot(self.P, self.Q)


### c - Toying Around : Testing it on Collaborative Filtering

Before scaling up the model, let's first test it on a toy dataset. We give you this R matrix. As you can see the R matrix contains some outliers (Nan values)
* How can we handle the outliers in this R matrix, what changes should you do ?
* Define a function that handles the outliers of your input matrix.

In [ ]:
# Not TODO : Define a Rating Matrix. An example of Rating Matrix is given
R = np.array([[np.NaN,5,1,0,4],
              [0,0,1,0,3],
              [0,5,0,2,np.NaN],
              [1,np.NaN,0,5,1] ])

# TODO : Define a function that handles the outliers of the R matrix. Hint nan_to_num 
def correct_outliers(R):
    return ...

# TODO : Verify that your function works
R = ...

In [ ]:
# TODO : Create an instance of the Matrix_Factorization Class
latent_dim = ...
MF = Matrix_Factorization(A = ...,
                          latent_dim=...)

In [ ]:
# TODO : Calculate the approximated Rating Matrix using the run method
mat = ...

### d - Scaling Up : Testing on the Luxury Dataset

Now let's test our model on the Luxury Dataset. In order to check the capacity of the algorithm. We masked few ratings to Nan values. The goal is to check if the model would be able to recapture the original rating.

The mf_df_train dataframe corresponds to the mf_df dataset but with some nan values in the Rating column. 
* Create a Rating Matrix using 'UserId' as Index and 'ProductId' as Columns.
* Initialize well the values of the Rating Matrix and convert it to numpy array.
* Run Matrix Factorization on the given Matrix for a feature space of size 2

In [ ]:
# TODO : Run Matrix Factorization for a latent space of dimension 2 
latent_dim = ...
MF = Matrix_Factorization(R,latent_dim)
R_pred = MF.run()

Now let's answer the following questions : 
* Will user 6 like item 9 ?
* Will user 4 like item 2 ?
* What do you think of this model ? (Is it fast ? Is it efficient ? Does it need lot of engineering ?)

# Hands-On III : Introducing Lightning with Recommendation Systems

Using MovieLens Dataset, build a recommendation system. Feel free for your model but we highly, suggest you to inspire yourself from the NCF Model. This part is **voluntarily sparse**. As future engineers, you'll need to go fetch informations everywhere in order to create and train your model. 



## I - Load, Analyze and Transform MovieLens Dataset


### a - Just do It

Using Panda Dataframe have a look at the ratings.csv and movies.csv files and answer the following questions.

* What are the features of both csv files ?
* To you, what could be the most important csv file ?
* How many movies are there ?
* How many users rated movies ? 
* How users did rate the movies ? 
* What's the span of the ratings ? Using matplotlib or seaborn, plot the ratings for a given movie.
* Can we merge both tables ? if yes what key should be used to merge them ?
* What are the differents genres of the movies ?
* Are there outliers ?


In [ ]:
# TODO : Perform your Exploratory Data Analysis
ratings = 
movies = 

### b - Some Changes

For the rest of the lab, we will only use .05% of the dataset as it it too much for this notebook ( RIP Google ). Run the following cell for the rating dataframe. It will replace the existing rating dataset by a splitted dataset corresponding to 0.05% of the dataset

In [ ]:
ratings, _ = train_test_split(ratings, test_size = 0.95)

### c - Create your Dataset class
At this point, you should have a good understanding of the given dataset.As we work with Tensors and Pytorch, we need to convert the Dataset to something readable by a Pytorch Model. Define a Dataset class that inherits from Dataset that returns a dictionnary {user, item, ratings}. The skeleton is given.
Don't forget to create your train, test, validation datasets.

Documentation available on : https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
class MovieLensDataset(Dataset):

  def __init__(self,dataframe):
      self.dataframe = 

  def __getitem__(self, idx):


      return {'user': torch.tensor(, dtype=torch.long),
              'movie':  torch.tensor(, dtype=torch.long),
              'rating':  torch.tensor(, dtype=torch.float)}

  def __len__(self):
    return len()


### d - Create your DataLoaders

Now as in the labs, we want to fetch lot of data at the same time to send it to the model. We use DataLoaders. Define Dataloaders for each of your DataSets

In [ ]:
dataloader_train = ...
dataloader_valid = ...

## II - Create a Model 

Create a model that inherits from pl.Lightning. On the contrairy of PyTorch nn.Module, a Lihgnting Model encompasses the training, validation steps. A skeleton is given. If you don't have any ideas, try to reimplement NCF model.

Documentation available on : https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [ ]:
class NCF(pl.LightningModule):

    def __init__(self, num_users, num_items, embedding_dim=32, hidden_dim=64):
        super().__init__()
        self.accuracy = torchmetrics.Accuracy()
        # TODO : Create your model

    def forward(self, ...):
        # TODO : Send what you need in your model

        return ...

    def training_step(self, batch, batch_idx):
        # TODO : Define your training step
        user_ids, item_ids, ratings = ...


        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
       # TODO : Define your validation step, don't forget to compute your accuracy
        user_ids, item_ids, ratings = ...

        self.log("val_loss", loss)

    def validation_epoch_end(self, outputs):
        self.log('val_accuracy', self.accuracy.compute())

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

## III - Trainer

Obviously, you can rewrite the same writing loop everytime, but we can be smarter and create a Trainer, that already has everything written in it. 

More Documentations on: https://lightning.ai/docs/pytorch/stable/common/trainer.html


### a - Set Up Tensorboard for Monitoring

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ...

### b - Create your trainer

In [ ]:
trainer = pl.Trainer(gpus=-1, 
                     max_epochs=..)

### c -  Train




In [ ]:
trainer.fit(..., 
            ...)


### d - Test

Test your model on the test set. 
